In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [ ]:
df = pd.read_csv('../data/cleaned_amex_data.csv')


df['event_ts'] = pd.to_datetime(df['event_ts'])
df['event_dt'] = pd.to_datetime(df['event_dt'])

print("Data loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Date range: {df['event_dt'].min()} to {df['event_dt'].max()}")

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv('../data/cleaned_amex_data.csv')

print("Dataset Shape:", df.shape)
print(df.info())


df['event_ts']=pd.to_datetime(df['event_ts'])
df['event_dt']=pd.to_datetime(df['event_dt'])

#date range of the dataset
print(f"\nDate Range: {df['event_dt'].min()} to {df['event_dt'].max()}")
print(f"Analysis Period: {(df['event_dt'].max() - df['event_dt'].min()).days} days")

#define activer  users

def identify_active_users(df, reference_date):
    
    print("Identify active users who visited Amex Portal at least once in the past 30 days from the reference date")
    
    cutoff_date=reference_date - timedelta(days=30)
    active_users=df[df['event_dt'] >= cutoff_date]['customer_id'].unique()
    return set(active_users)

def get_airline_offers(df):
    
    print("Identify airline offers based on var_50 (travel category) and var_9 (airlines interest score")
    
    #travel category(var_50=1) or have high airline interest(var_9)
    airline_offers = df[
        (df['var_50'] == 1) |  # Travel category
        (df['var_9'].notna() & (df['var_9'] > df['var_9'].median()))  # High airline interest
    ]
    return airline_offers

def calculate_discount_savings(row):
    
    #calculate savings from discounts: discount value (var_34) and discount rate (var_33)
    
    if pd.notna(row['var_34']):
        return row['var_34']  # Direct discount value
    elif pd.notna(row['var_33']) and pd.notna(row['var_32']):
        return row['var_32'] * row['var_33']  # Min spend * discount rate
    else:
        return 0


Q.1.1 Active users are more likely to choose Airline Offers 

In [ ]:
#get the last date in dataset as reference
reference_date= df['event_dt'].max()
active_users= identify_active_users(df, reference_date)

print(f"Total unique customers: {df['customer_id'].nunique()}")
print(f"Active users (last 30 days): {len(active_users)}")
print(f"Active users percentage: {len(active_users)/df['customer_id'].nunique()*100:.2f}%")

#create user activity flag
df['is_active_user'] =df['customer_id'].isin(active_users)

#offer categories based on one-hot encoded variables
offer_categories= {
    'Business': 'var_44',
    'Dining': 'var_45', 
    'Entertainment': 'var_46',
    'Retail': 'var_47',
    'Services': 'var_48',
    'Shopping': 'var_49',
    'Travel': 'var_50'
}

#calculate preference rates for each category
print("\nOffer Category Preferences:")


category_preferences= {}
for category, var_name in offer_categories.items():
    #ttal clicks in this category
     total_clicks = df[(df[var_name] == 1) & (df['offer_action'] == 'click')].shape[0]

    #zctive user clicks in this category
     active_clicks = df[(df[var_name] == 1) & (df['is_active_user']) & (df['offer_action'] == 'click')].shape[0]

    #non-active user clicks in this category
     non_active_clicks = df[(df[var_name] == 1) & (~df['is_active_user']) & (df['offer_action'] == 'click')].shape[0]
    
    #click rates
     active_users_in_category=df[(df[var_name]==1) & (df['is_active_user'])]['customer_id'].nunique()
     non_active_users_in_category = df[(df[var_name]==1) & (~df['is_active_user'])]['customer_id'].nunique()
    
     active_click_rate=active_clicks / max(active_users_in_category, 1)
     non_active_click_rate = non_active_clicks / max(non_active_users_in_category, 1)
    
     category_preferences[category]={
        'active_click_rate': active_click_rate,
        'non_active_click_rate': non_active_click_rate,
        'total_clicks': total_clicks
    }
    
     print(f"{category}: Active Users Click Rate={active_click_rate:.4f}, "
          f"Non-Active Users Click Rate={non_active_click_rate:.4f}")

#check if active users prefer airline/travel offers more var50
travel_pref=category_preferences['Travel']
airline_preference_higher =travel_pref['active_click_rate'] > travel_pref['non_active_click_rate']

print(f"\nQUESTION 1.1:")
print(f"Active users prefer airline/travel offers more than non-active users: {airline_preference_higher}")


Q.1.2 Active Users vs Discount Savings

In [ ]:
#calculate discount savings for each transaction
df['discount_savings']=df.apply(calculate_discount_savings, axis=1)

#filter for transactions with actual discounts
discount_transactions=df[df['discount_savings'] > 0].copy()

print(f"Total transactions with discounts: {len(discount_transactions)}")

if len(discount_transactions) > 0:
    #calculate average savings by user type
    active_user_savings = discount_transactions[discount_transactions['is_active_user']]['discount_savings']
    non_active_user_savings = discount_transactions[~discount_transactions['is_active_user']]['discount_savings']
    
    print(f"Active users with discount transactions: {len(active_user_savings)}")
    print(f"Non-active users with discount transactions: {len(non_active_user_savings)}")
    
    if len(active_user_savings) > 0 and len(non_active_user_savings) >= 0:
        avg_savings_active = active_user_savings.mean()
        avg_savings_non_active = non_active_user_savings.mean()
        
        print(f"\nAverage discount savings:")
        print(f"Active users: ${avg_savings_active:.2f}")
        print(f"Non-active users: ${avg_savings_non_active:.2f}")
        
        #statistical test
        t_stat, p_value = stats.ttest_ind(active_user_savings, non_active_user_savings)
        
        print(f"\nStatistical Test (t-test):")
        print(f"T-statistic: {t_stat:.4f}")
        print(f"P-value: {p_value:.4f}")
        
        savings_difference= avg_savings_active > avg_savings_non_active
        statistically_significant= p_value < 0.05
        
        print(f"\nQUESTION 1.2:")
        print(f"Active users save more money from discounts: {savings_difference}")
        print(f"Difference is statistically significant (p<0.05): {statistically_significant}")

        
        


Q.1.3 Total Population of IDs out of total customers who actively use Amex Portal and have clicked on an Airline Offer

In [ ]:
print(df['offer_action'].unique())

In [ ]:
#identify airline offers var50
airline_offers = df[df['var_50'] == 1].copy()

print(f"Total travel/airline offers in dataset: {len(airline_offers)}")

#active users who clicked on airline offers
active_users_airline_clicks = airline_offers[
    (airline_offers['is_active_user']) & 
    (airline_offers['offer_action'].isin(['1', 'True']))
]['customer_id'].nunique()


#total unique customers in the dataset
total_unique_customers = df['customer_id'].nunique()

#active users who clicked on any airline offer
active_users_with_airline_clicks = airline_offers[
    (airline_offers['is_active_user']) & 
    (airline_offers['offer_action'] == 1)
]['customer_id'].unique()

print(f"\n Analysis:")
print(f"Total unique customers in dataset: {total_unique_customers:,}")
print(f"Active users in dataset: {len(active_users):,}")
print(f"Unique active users who clicked on airline offers: {active_users_airline_clicks:,}")

#Calculate percentages
pct_active_users = (len(active_users) / total_unique_customers) * 100
pct_active_airline_clickers = (active_users_airline_clicks / len(active_users)) * 100 if len(active_users) > 0 else 0
pct_airline_clickers_of_total = (active_users_airline_clicks / total_unique_customers) * 100



print(f"\nQUESTION 1.3:")
print(f"Total population of unique customer IDs who actively use Amex Portal and clicked on Airline Offers: {active_users_airline_clicks:,}")



QUESTION 2

Q.2.1

In [ ]:
#check the discount value per dollar variable (var_37)
print("Discount Value per Dollar (var_37) Statistics:")
print(df['var_37'].describe())
print(f"\nMissing values in var_37: {df['var_37'].isnull().sum()}")
print(f"Zero values in var_37: {(df['var_37'] == 0).sum()}")

#exclude offers with no Discount Value per Dollar
df_filtered = df[df['var_37'].notna() & (df['var_37'] > 0)].copy()

print(f"\nOriginal dataset size: {len(df)}")
print(f"After excluding offers with no discount value per dollar: {len(df_filtered)}")

if len(df_filtered) ==0:
    print("ERROR: No data available after filtering for discount value per dollar > 0")
    exit()

#50th percentile (median) of discount value per dollar
percentile_50 =df_filtered['var_37'].quantile(0.5)
print(f"\n50th Percentile of Discount Value per Dollar: {percentile_50:.6f}")

#high/low discount value per dollar categories
df_filtered['disc_value_category'] = np.where(
    df_filtered['var_37'] >= percentile_50, 
    'High', 
    'Low'
)

print(f"\nHigh Discount Value per Dollar offers: {(df_filtered['disc_value_category'] == 'High').sum()}")
print(f"Low Discount Value per Dollar offers: {(df_filtered['disc_value_category'] == 'Low').sum()}")


#categories mentioned
categories ={
    'Services': 'var_48',
    'Shopping': 'var_49', 
    'Travel': 'var_50'
}


def calculate_conversion_rate(data):
    
    if len(data) == 0:
        return 0.0
    
    offer_actions_numeric = data['offer_action'].astype(str).str.strip().map({'1': 1, 'True': 1, '0': 0, 'False': 0}).fillna(0)
    return offer_actions_numeric.sum() / len(data)


def floor_to_3_decimals(value):
    
    return np.floor(value * 1000) / 1000


results = {}

for category_name, var_name in categories.items():
    print(f"{category_name.upper()} ({var_name})")
    
    
    category_data =df_filtered[df_filtered[var_name] == 1].copy()
    
    print(f"Total offers in {category_name} category: {len(category_data)}")
    
    if len(category_data)== 0:
        print(f"No data available for {category_name} category")
        results[category_name] ={
            'high_conversion_rate': 0.0,
            'low_conversion_rate': 0.0,
            'difference': 0.0,
            'difference_floored': 0.0
        }
        continue
    
    #high low discount value categories
    high_disc_data = category_data[category_data['disc_value_category'] == 'High']
    low_disc_data = category_data[category_data['disc_value_category'] == 'Low']
    
    print(f"High Discount Value offers in {category_name}: {len(high_disc_data)}")
    print(f"Low Discount Value offers in {category_name}: {len(low_disc_data)}")
    
    #conversion rates
    high_conversion_rate = calculate_conversion_rate(high_disc_data)
    low_conversion_rate = calculate_conversion_rate(low_disc_data)

    
    print(f"\nConversion Rates:")
    print(f"High Discount Value per Dollar: {high_conversion_rate:.6f}")
    print(f"Low Discount Value per Dollar: {low_conversion_rate:.6f}")
    
    #difference
    difference = high_conversion_rate - low_conversion_rate
    difference_floored = floor_to_3_decimals(difference)
    
    print(f"\nDifference [High - Low]: {difference:.6f}")
    print(f"Difference (floored to 3 decimals): {difference_floored:.3f}")
    
    #results
    results[category_name] = {
    'high_conversion_rate': high_conversion_rate,
    'low_conversion_rate': low_conversion_rate,
    'difference': difference,
    'difference_floored': difference_floored,
    'high_count': len(high_disc_data),
    'low_count': len(low_disc_data),
    'high_conversions': high_disc_data['offer_action'].astype(str).str.strip().map({'1': 1, 'True': 1, '0': 0, 'False': 0}).fillna(0).sum() if len(high_disc_data) > 0 else 0,
    'low_conversions': low_disc_data['offer_action'].astype(str).str.strip().map({'1': 1, 'True': 1, '0': 0, 'False': 0}).fillna(0).sum() if len(low_disc_data) > 0 else 0
}


#final
print(f"\nDiscount Value per Dollar 50th Percentile Threshold: {percentile_50:.6f}")
print(f"Analysis based on {len(df_filtered)} offers with valid discount values")

print(f"\n{'Category':<15} {'High Rate':<12} {'Low Rate':<12} {'Difference':<12} {'Floored':<10}")

final_answers = {}

for category_name in ['Services', 'Shopping', 'Travel']:
    result = results[category_name]
    high_rate = result['high_conversion_rate']
    low_rate = result['low_conversion_rate']
    difference = result['difference']
    floored_diff = result['difference_floored']
    
    print(f"{category_name:<15} {high_rate:<12.6f} {low_rate:<12.6f} {difference:<12.6f} {floored_diff:<10.3f}")
    
    final_answers[category_name] = floored_diff


print(f"\n1. Services: {final_answers['Services']:.3f}")
print(f"2. Shopping: {final_answers['Shopping']:.3f}") 
print(f"3. Travel: {final_answers['Travel']:.3f}")



QUESTION 3

In [ ]:
import math 
def analyze_customer_spending_and_rewards(df):
    
    #1: calculate maximum spending for each customer in last 3 months
    spending_cols = ['var_17', 'var_18', 'var_19']  # Lifestyle, Electronics, Restaurant spend

    #for each customer, calculate the maximum spend across all categories in last 3 months
    customer_spend = df.groupby('customer_id')[spending_cols].sum()
    customer_max_spend = customer_spend.max(axis=1).reset_index()
    customer_max_spend.columns = ['customer_id', 'max_spend_3m']

    #2: divide customers into Low, Medium, High spending categories with equal counts
    
    customer_max_spend['spending_category'] = pd.qcut(customer_max_spend['max_spend_3m'], q=3, labels=['LOW', 'MEDIUM', 'HIGH'])

    #3: mrge spending categories back to main dataframe
    df_with_categories = df.merge(customer_max_spend[['customer_id', 'spending_category']],
                                  on='customer_id', how='left')

    #4: calculate Average Reward Point to Dollar Value Conversion Rate for offers offered to each category
    offered_conversion_rates = df_with_categories.groupby('spending_category')['var_36'].mean()

    #5: calculate Average Reward Point to Dollar Value Conversion Rate for participated offers
    participated_df = df_with_categories[df_with_categories['offer_action'].notna()]
    participated_conversion_rates = participated_df.groupby('spending_category')['var_36'].mean()

    #6: calculate participation rate for each category
    total_offers_per_category = df_with_categories.groupby('spending_category').size()
    participated_offers_per_category = participated_df.groupby('spending_category').size()
    participation_rates = (participated_offers_per_category / total_offers_per_category).fillna(0)

    #7: dtermine which segment performed best (highest participation rate)
    best_segment = participation_rates.idxmax()

    def floor_to_3_decimal(value):
        return math.floor(value * 1000) / 1000

    #res
    results= {
        'offered_conversion_rates': {
            'LOW': floor_to_3_decimal(offered_conversion_rates.get('LOW', 0)),
            'MEDIUM': floor_to_3_decimal(offered_conversion_rates.get('MEDIUM', 0)),
            'HIGH': floor_to_3_decimal(offered_conversion_rates.get('HIGH', 0))
        },
        'participated_conversion_rates': {
            'LOW': floor_to_3_decimal(participated_conversion_rates.get('LOW', 0)),
            'MEDIUM': floor_to_3_decimal(participated_conversion_rates.get('MEDIUM', 0)),
            'HIGH': floor_to_3_decimal(participated_conversion_rates.get('HIGH', 0))
        },
        'participation_rates': {
            'LOW': participation_rates.get('LOW', 0),
            'MEDIUM': participation_rates.get('MEDIUM', 0),
            'HIGH': participation_rates.get('HIGH', 0)
        },
        'best_performing_segment': best_segment,
        'customer_distribution': {
            'LOW': sum(customer_max_spend['spending_category'] == 'LOW'),
            'MEDIUM': sum(customer_max_spend['spending_category'] == 'MEDIUM'),
            'HIGH': sum(customer_max_spend['spending_category'] == 'HIGH')
        }
    }

    return results

def print_results(results):
            
    print("1. Customer Distribution Across Categories:")
    for category, count in results['customer_distribution'].items():
        print(f"   {category}: {count} customers")
    
    print("\n2. Average Reward Point to Dollar Conversion Rate - All Offers:")
    for category, rate in results['offered_conversion_rates'].items():
        print(f"   {category}: {rate:.3f}")
    
    print("\n3. Average Reward Point to Dollar Conversion Rate - Participated Offers:")
    for category, rate in results['participated_conversion_rates'].items():
        print(f"   {category}: {rate:.3f}")
    
    print("\n4. Participation Rates by Category:")
    for category, rate in results['participation_rates'].items():
        print(f"   {category}: {rate:.3%}")
    
    print(f"\n5. Best Performing Segment: {results['best_performing_segment']}")
    
    print("\n final")
    print("Answer 1 :")
    print(f"LOW: {results['offered_conversion_rates']['LOW']:.3f}")
    print(f"MEDIUM: {results['offered_conversion_rates']['MEDIUM']:.3f}")
    print(f"HIGH: {results['offered_conversion_rates']['HIGH']:.3f}")
    
    print("\nAnswer 2 :")
    print(f"LOW: {results['participated_conversion_rates']['LOW']:.3f}")
    print(f"MEDIUM: {results['participated_conversion_rates']['MEDIUM']:.3f}")
    print(f"HIGH: {results['participated_conversion_rates']['HIGH']:.3f}")
    
    print(f"\nAnswer 3 : {results['best_performing_segment']}")

results = analyze_customer_spending_and_rewards(df)
print_results(results)



QUESTION 4:

In [ ]:
def analyze_top_offers_by_conversion(df):
    
    if 'event_dt' in df.columns:
        df['event_dt'] = pd.to_datetime(df['event_dt'])
    
    #Reference date 7 Nov 2023
    reference_date=datetime(2023, 11, 7)
    
    #1: calculate offer conversion rates
    #conversion rate=no of Offers Clicked /no. of Offers Viewed
    
        
    #views and clicks per offer
    offer_stats = df.groupby('offer_id').agg({
        'var_40': 'sum',  # Rolling Impressions (views)
        'var_39': 'sum',  # Rolling Clicks
        'var_43': 'sum',  # No of impressions on the offer (in last 3 days)
        'customer_id': 'count'  # Total records per offer
    }).reset_index()
    
    #conversion rate using available metrics
    #use rolling clicks / rolling impressions
    offer_stats['impressions'] = offer_stats['var_40']
    offer_stats['clicks'] = offer_stats['var_39']
    
    
    offer_stats['impressions'] = offer_stats['impressions'].fillna(offer_stats['var_43'])
    
    #conversion rate
    offer_stats['conversion_rate'] = np.where(
        offer_stats['impressions'] > 0,
        offer_stats['clicks'] / offer_stats['impressions'],
        0
    )
    
    #2: filter offers presented above average presentation count
    avg_presentation_count = offer_stats['impressions'].mean()
    qualified_offers = offer_stats[offer_stats['impressions'] >= avg_presentation_count]
    
    # Step 3: Get top 10 offers by conversion rate
    top_10_offers = qualified_offers.nlargest(10, 'conversion_rate')
    
    # Step 4: Create offer category mappings
    # Extract offer categories from one-hot encoded variables (var_44 to var_50)
    category_mapping = {}
    category_columns = {
        'var_44': 'Business',
        'var_45': 'Dining', 
        'var_46': 'Entertainment',
        'var_47': 'Retail',
        'var_48': 'Services',
        'var_49': 'Shopping',
        'var_50': 'Travel'
    }
    
    for offer_id in top_10_offers['offer_id']:
        offer_data = df[df['offer_id'] == offer_id]
        
        #category with highest value (assuming one-hot encoding)
        category_scores = {}
        for var, category in category_columns.items():
            if var in offer_data.columns:
                category_scores[category] = offer_data[var].max()
        
        #highest score
        if category_scores:
            assigned_category = max(category_scores, key=category_scores.get)
            category_mapping[offer_id] = assigned_category
        else:
            category_mapping[offer_id] = 'Unknown'
    
    #5: average 3-month spend for customers opting for these offers as of nov 7
    #filterdata up to reference date
    df_filtered = df[df['event_dt'] <= reference_date] if 'event_dt' in df.columns else df
    
    #get customers who opted for top 10 offers
    top_offer_customers = df_filtered[df_filtered['offer_id'].isin(top_10_offers['offer_id'])]
    
    #3-month spend
    spending_cols = ['var_17', 'var_18', 'var_19']
    top_offer_customers['total_3m_spend'] = top_offer_customers[spending_cols].sum(axis=1)
    
    #avg 3 month
    avg_3m_spend = top_offer_customers['total_3m_spend'].mean()
    
    #6: calculate average conversion rate for customers enrolling for each offer over last 30 days
    #filter last 30 days from reference date
    thirty_days_ago = reference_date - timedelta(days=30)
    df_30d = df_filtered[df_filtered['event_dt'] >= thirty_days_ago] if 'event_dt' in df.columns else df_filtered
    
    #conversion rates for each top offer in last 30 days
    offer_30d_conversion = {}
    
    for offer_id in top_10_offers['offer_id']:
        offer_30d_data = df_30d[df_30d['offer_id'] == offer_id]
        
        if len(offer_30d_data) > 0:
            #click-through rate or calculate from available data
            if 'var_41' in offer_30d_data.columns:
                #existing click through rate
                conversion_rate_30d = offer_30d_data['var_41'].mean()
            else:
                #
                total_impressions = offer_30d_data['var_40'].sum()
                total_clicks = offer_30d_data['var_39'].sum()
                conversion_rate_30d = total_clicks / total_impressions if total_impressions > 0 else 0
        else:
            conversion_rate_30d = 0
        
        offer_30d_conversion[offer_id] = conversion_rate_30d
    

    def floor_to_3_decimal(value):
        return math.floor(value * 1000) / 1000
    

    results = {
        'top_10_offers': top_10_offers[['offer_id', 'conversion_rate', 'impressions', 'clicks']].to_dict('records'),
        'offer_category_mapping': category_mapping,
        'avg_3m_spend': floor_to_3_decimal(avg_3m_spend),
        'offer_30d_conversion_rates': {
            offer_id: floor_to_3_decimal(rate) 
            for offer_id, rate in offer_30d_conversion.items()
        },
        'avg_presentation_count': avg_presentation_count,
        'qualified_offers_count': len(qualified_offers)
    }
    
    return results

def print_offer_analysis_results(results):
    """Print the offer analysis results in a formatted manner"""
    
    print("top 10\n")
    
    print(f"Average Presentation Count Threshold: {results['avg_presentation_count']:.2f}")
    print(f"Number of Qualified Offers: {results['qualified_offers_count']}")
    
    print("\n1. Top 10 Offer ID to Offer Category Mappings:")
    for i, offer in enumerate(results['top_10_offers'], 1):
        offer_id = offer['offer_id']
        category = results['offer_category_mapping'].get(offer_id, 'Unknown')
        conversion_rate = offer['conversion_rate']
        print(f"   {i}. {offer_id}: {category} (Conversion Rate: {conversion_rate:.4f})")
    
    print(f"\n2. Average 3-Month Spend of Customers (as of Nov 7, 2023): {results['avg_3m_spend']:.3f}")
    
    print("\n3. Average Conversion Rate for Each Offer (Last 30 Days as of Nov 7, 2023):")
    for offer_id, rate in results['offer_30d_conversion_rates'].items():
        print(f"   {offer_id}: {rate:.3f}")
    
    print("final")
    print("Answer 1 (Top 10 Offer ID: Category Mappings):")
    for offer in results['top_10_offers']:
        offer_id = offer['offer_id']
        category = results['offer_category_mapping'].get(offer_id, 'Unknown')
        print(f"{offer_id}: {category}")
    
    print(f"\nAnswer 2 (Average 3-Month Spend): {results['avg_3m_spend']:.3f}")
    
    print("\nAnswer 3 (Offer ID: 30-Day Conversion Rates):")
    for offer_id, rate in results['offer_30d_conversion_rates'].items():
        print(f"{offer_id}: {rate:.3f}")

results = analyze_top_offers_by_conversion(df)


print_offer_analysis_results(results)


QUESTION 5


In [ ]:
from datetime import datetime, timedelta

def analyze_top_customers_email_campaigns(df):
  
    if 'event_dt' in df.columns:
        df['event_dt'] = pd.to_datetime(df['event_dt'])
    
    #reference date 14 Nov 2023
    reference_date = datetime(2023, 11, 14)
    thirty_days_ago = reference_date - timedelta(days=30)
    
    #filter data for last 30 days as of 14 Nov 2023
    if 'event_dt' in df.columns:
        df_30d = df[(df['event_dt'] >= thirty_days_ago) & (df['event_dt'] <= reference_date)]
    else:
        # 
        df_30d = df.copy()
    
    #customers who participated in email campaigns in past 30 days
    # var_14: Number of successfully sent emails in the last 30 days
    # var_15: Number of emails clicked on in the last 30 days
    #filter customers with email activity
    email_participants = df_30d[
        (df_30d['var_14'] > 0) | (df_30d['var_15'] > 0)
    ]['customer_id'].unique()
    
    #filter dataset to only include email campaign participants
    df_email_participants = df_30d[df_30d['customer_id'].isin(email_participants)]
    
    #offer conversion rate for each customer
    #Conversion rate = Number of Offers Clicked / Number of Offers Viewed given
    #var_39 (Rolling Clicks) var_40 (Rolling Impressions)
    
    customer_conversion_stats = df_email_participants.groupby('customer_id').agg({
        'var_39': 'sum',  # Total rolling clicks
        'var_40': 'sum',  # Total rolling impressions (views)
        'var_43': 'sum',  # Alternative impressions measure
        'var_15': 'first', # Email clicks in last 30 days
        'var_14': 'first', # Emails sent in last 30 days
        'var_13': 'first'  # Distinct channels used in last 30 days copied from data dictionary
    }).reset_index()
    
    #var_40 as primary, var_43 as fallback for impressions
    customer_conversion_stats['total_impressions'] = np.where(
        customer_conversion_stats['var_40'] > 0,
        customer_conversion_stats['var_40'],
        customer_conversion_stats['var_43']
    )
    
    customer_conversion_stats['offer_conversion_rate'] = np.where(
        customer_conversion_stats['total_impressions'] > 0,
        customer_conversion_stats['var_39'] / customer_conversion_stats['total_impressions'],
        0
    )
    
    #top 15 customers by offer conversion rate
    top_15_customers = customer_conversion_stats.nlargest(15, 'offer_conversion_rate')
    
    #ratio of total offer clicks to distinct channels used
    #Ratio = Total Offer Clicks / Count of Distinct Channels???????
    #var_39: Total offer clicks, var_13: Distinct channels used >>copied
    
    top_15_customers['clicks_to_channels_ratio'] = np.where(
        top_15_customers['var_13'] > 0,
        top_15_customers['var_39'] / top_15_customers['var_13'],
        0
    )
    
    #top-performing offer category for each customer
    #var_44 to var_50 copied??
    category_columns = {
        'var_44': 'Business',
        'var_45': 'Dining', 
        'var_46': 'Entertainment',
        'var_47': 'Retail',
        'var_48': 'Services',
        'var_49': 'Shopping',
        'var_50': 'Travel'
    }
    
    top_performing_categories = {}
    
    for customer_id in top_15_customers['customer_id']:
        customer_data = df_email_participants[df_email_participants['customer_id'] == customer_id]
        
        #total engagement/clicks per category
        category_performance = {}
        for var, category in category_columns.items():
            if var in customer_data.columns:
                
                category_clicks = (customer_data[var] * customer_data['var_39']).sum()
                category_performance[category] = category_clicks
        
        #top performing category
        if category_performance:
            top_category = max(category_performance, key=category_performance.get)
            top_performing_categories[customer_id] = top_category
        else:
            top_performing_categories[customer_id] = 'Unknown'
    
    #email offer click to email sent ratio
    # Ratio = Email Clicks / Email Sent coiped?
    # var_15: Email clicks, var_14: Emails sent copied???
    
    top_15_customers['email_click_to_sent_ratio'] = np.where(
        top_15_customers['var_14'] > 0,
        top_15_customers['var_15'] / top_15_customers['var_14'],
        0
    )
    
    def round_to_3_decimal(value):
       if pd.isna(value):
         return 0.0
       return round(value, 3)


    
    results = {
        'top_15_customers': top_15_customers[['customer_id', 'offer_conversion_rate']].to_dict('records'),
        'clicks_to_channels_ratio': {
            row['customer_id']: round_to_3_decimal(row['clicks_to_channels_ratio'])
            for _, row in top_15_customers.iterrows()
        },
        'top_performing_categories': top_performing_categories,
        'email_click_to_sent_ratio': {
    row['customer_id']: round_to_3_decimal(row['email_click_to_sent_ratio'])
    for _, row in top_15_customers.iterrows()
},
        'email_participants_count': len(email_participants),
        'analysis_date_range': f"{thirty_days_ago.strftime('%Y-%m-%d')} to {reference_date.strftime('%Y-%m-%d')}"
    }
    
    return results

def analyze_top_customers_enhanced(df):
    category_columns = {
        'var_44': 'Business',
        'var_45': 'Dining', 
        'var_46': 'Entertainment',
        'var_47': 'Retail',
        'var_48': 'Services',
        'var_49': 'Shopping',
        'var_50': 'Travel'
    }

    customer_category_performance = []
    
    for customer_id in df['customer_id'].unique():
        customer_data = df[df['customer_id'] == customer_id]
        
        for var, category in category_columns.items():
            if var in customer_data.columns:
                # sum
                category_total_clicks = (customer_data[var] * customer_data['var_39']).sum()
                if category_total_clicks > 0:
                    customer_category_performance.append({
                        'customer_id': customer_id,
                        'category': category,
                        'total_clicks': category_total_clicks
                    })
    
    # convert to datframe easier
    category_df = pd.DataFrame(customer_category_performance)
    if not category_df.empty:
        top_categories_per_customer = category_df.loc[
            category_df.groupby('customer_id')['total_clicks'].idxmax()
        ][['customer_id', 'category']].set_index('customer_id')['category'].to_dict()
    else:
        top_categories_per_customer = {}
    
    return analyze_top_customers_email_campaigns(df)

def print_customer_analysis_results(results):
    """Print the customer analysis results in a formatted manner"""
    
    print("top 15\n")
    
    print(f"Analysis Period: {results['analysis_date_range']}")
    print(f"Total Email Campaign Participants: {results['email_participants_count']}")
    
    print("\nTop 15 Customers by Offer Conversion Rate:")
    for i, customer in enumerate(results['top_15_customers'], 1):
        customer_id = customer['customer_id']
        conversion_rate = customer['offer_conversion_rate']
        print(f"   {i}. Customer {customer_id}: {conversion_rate:.4f}")
    
    print("\n1. Ratio of Total Offer Clicks to Distinct Channels (CustomerID: Ratio):")
    for customer_id, ratio in results['clicks_to_channels_ratio'].items():
        print(f"   {customer_id}: {ratio:.3f}")
    
    print("\n2. Top-Performing Offer Category per Customer (CustomerID: Category):")
    for customer_id, category in results['top_performing_categories'].items():
        print(f"   {customer_id}: {category}")
    
    print("\n3. Email Click to Email Sent Ratio (CustomerID: Ratio):")
    for customer_id, ratio in results['email_click_to_sent_ratio'].items():
        print(f"   {customer_id}: {ratio:.3f}")
    
    print("\nfinal")
    print("Answer 1 :")
    for customer_id, ratio in results['clicks_to_channels_ratio'].items():
        print(f"{customer_id}: {ratio:.3f}")
    
    print("\nAnswer 2 :")
    for customer_id, category in results['top_performing_categories'].items():
        print(f"{customer_id}: {category}")
    
    print("\nAnswer 3 :")
    for customer_id, ratio in results['email_click_to_sent_ratio'].items():
        print(f"{customer_id}: {ratio:.3f}")

results = analyze_top_customers_email_campaigns(df)
print_customer_analysis_results(results)


QUESTION 6

In [ ]:

def calculate_email_offer_probabilities(df):
     
    if 'event_dt' in df.columns:
        df['event_dt'] = pd.to_datetime(df['event_dt'])
        
        df = df.sort_values(['customer_id', 'event_dt']).reset_index(drop=True)
    else:
        #sort by event_ts
        if 'event_ts' in df.columns:
            df = df.sort_values(['customer_id', 'event_ts']).reset_index(drop=True)
    
    #calculate the probability of clicking on marketing email offer being sent out
    #var_14: Number of successfully sent emails in the last 30 days
    #var_15: Number of emails clicked on in the last 30 days
    
    #customer-level email statistics
    customer_email_stats = df.groupby('customer_id').agg({
        'var_14': 'first',  # Emails sent
        'var_15': 'first',  # Email clicks 
        'event_dt': 'count' if 'event_dt' in df.columns else lambda x: len(x)  # Number of records
    }).reset_index()
    
    #basic click rate for each customer
    customer_email_stats['basic_click_rate'] = np.where(
        customer_email_stats['var_14'] > 0,
        customer_email_stats['var_15'] / customer_email_stats['var_14'],
        0
    )
    
    #alpha=0.5
    #for each calculate EWMA-weighted click probability>>>recent
    
    def calculate_ewma_probability(customer_data, alpha=0.5):
        if len(customer_data) == 0:
            return 0
        
        #sort by date
        if 'event_dt' in customer_data.columns:
            customer_data = customer_data.sort_values('event_dt', ascending=False)
        elif 'event_ts' in customer_data.columns:
            customer_data = customer_data.sort_values('event_ts', ascending=False)
        
        #c click indicators
        click_indicators = []
        
        for _, row in customer_data.iterrows():
            email_click_rate = row['var_15'] / row['var_14'] if row['var_14'] > 0 else 0
            recent_click = row['var_31'] if 'var_31' in customer_data.columns else 0
            
            click_indicator = min(1.0, email_click_rate + (recent_click / max(1, row['var_14'])))
            click_indicators.append(click_indicator)
        
        if len(click_indicators) == 0:
            return 0
        
        ewma_value = click_indicators[0]
        for i in range(1, len(click_indicators)):
            ewma_value = alpha * click_indicators[i] + (1 - alpha) * ewma_value
        
        return ewma_value
    
    #probabilities for each customer
    ewma_probabilities = []
    
    for customer_id in df['customer_id'].unique():
        customer_data = df[df['customer_id'] == customer_id]
        ewma_prob = calculate_ewma_probability(customer_data, alpha=0.5)
        ewma_probabilities.append({
            'customer_id': customer_id,
            'ewma_probability': ewma_prob
        })
    
    ewma_df = pd.DataFrame(ewma_probabilities)
    
    #overall probability of clicking on marketing email offer
    overall_email_click_probability = ewma_df['ewma_probability'].mean()
    
    #conditional probability given user has clicked on an offer in the past
    customers_with_past_clicks = df[
        (df['var_39'] > 0) | (df['var_15'] > 0)
    ]['customer_id'].unique()
    
    ewma_past_clickers = ewma_df[ewma_df['customer_id'].isin(customers_with_past_clicks)]
    
    conditional_probability = ewma_past_clickers['ewma_probability'].mean() if len(ewma_past_clickers) > 0 else 0

    
    def floor_to_3_decimal(value):
        return math.floor(value * 1000) / 1000

    results = {
        'overall_email_click_probability': floor_to_3_decimal(overall_email_click_probability),
        'conditional_probability_given_past_clicks': floor_to_3_decimal(conditional_probability),
        'total_customers': len(df['customer_id'].unique()),
        'customers_with_past_clicks': len(customers_with_past_clicks),
        'average_basic_click_rate': customer_email_stats['basic_click_rate'].mean(),
        'ewma_statistics': {
            'mean': ewma_df['ewma_probability'].mean(),
            'std': ewma_df['ewma_probability'].std(),
            'min': ewma_df['ewma_probability'].min(),
            'max': ewma_df['ewma_probability'].max()
        }
    }
    
    return results

def enhanced_ewma_calculation(df):
       
    
    if 'event_dt' in df.columns:
        df_sorted = df.sort_values(['customer_id', 'event_dt']).reset_index(drop=True)
    else:
        df_sorted = df.sort_values(['customer_id', 'event_ts']).reset_index(drop=True)
    
    customer_ewma_probs = []
    
    for customer_id in df_sorted['customer_id'].unique():
        customer_data = df_sorted[df_sorted['customer_id'] == customer_id]
        
        click_events = []
        
        for _, row in customer_data.iterrows():
            
            email_rate = row['var_15'] / row['var_14'] if row['var_14'] > 0 else 0
            
            
            offer_engagement = min(1.0, row['var_39'] / max(1, row['var_40'])) if 'var_40' in customer_data.columns else 0
            
            
            recent_bonus = min(0.2, row['var_31'] / 10) if 'var_31' in customer_data.columns else 0
            
           
            event_prob = min(1.0, email_rate + offer_engagement + recent_bonus)
            click_events.append(event_prob)
        
        
        if click_events:
            ewma_prob = click_events[0]
            for i in range(1, len(click_events)):
                ewma_prob = 0.5 * click_events[i] + 0.5 * ewma_prob
            
            customer_ewma_probs.append(ewma_prob)
    
    return np.mean(customer_ewma_probs) if customer_ewma_probs else 0

def print_probability_analysis_results(results):
    """Print the probability analysis results in a formatted manner"""
    
    print("ANALYSIS\n")
    
    print(f"Total Customers Analyzed: {results['total_customers']}")
    print(f"Customers with Past Clicks: {results['customers_with_past_clicks']}")
    print(f"Average Basic Click Rate: {results['average_basic_click_rate']:.4f}")
    
    print("\nEWMA Statistics:")
    print(f"  Mean: {results['ewma_statistics']['mean']:.4f}")
    print(f"  Std:  {results['ewma_statistics']['std']:.4f}")
    print(f"  Min:  {results['ewma_statistics']['min']:.4f}")
    print(f"  Max:  {results['ewma_statistics']['max']:.4f}")
    
    print("\nfinal")
    
    print(f"\nANSWER 1: Probability of clicking on marketing email offer")
    print(f"         {results['overall_email_click_probability']:.3f}")
    
    print(f"\nANSWER 2: Conditional probability given past clicks")
    print(f"         {results['conditional_probability_given_past_clicks']:.3f}")

def validate_probability_data(df):

    required_columns = ['customer_id', 'var_14', 'var_15', 'var_39']
    optional_columns = ['var_31', 'var_40', 'var_41', 'event_dt', 'event_ts']
    
    missing_required = [col for col in required_columns if col not in df.columns]
    available_optional = [col for col in optional_columns if col in df.columns]
    
    if missing_required:
        print(f"Error: Missing required columns: {missing_required}")
        return False
    
    print(f"Available optional columns: {available_optional}")
    
    # Check email data quality
    email_data_quality = df[df['var_14'] > 0].shape[0] / df.shape[0]
    print(f"Percentage of records with email data: {email_data_quality:.2%}")
    
    return True
    
validate_probability_data(df)
results = calculate_email_offer_probabilities(df)


print_probability_analysis_results(results)
